# Configuración inicial y librerías

Se importan las librerías necesarias para llevar a cabo el preprocesamiento del dataset. Este conjunto de datos requiere transformación de fechas, codificación de variables categóricas, creación de nuevas características y manejo de valores faltantes. A continuación, se explican brevemente las librerías utilizadas:

- `pandas`: manipulación de datos estructurados.
- `numpy`: operaciones numéricas y matemáticas.
- `scipy.stats`: operaciones estadísticas como la moda.
- `sklearn.preprocessing.LabelEncoder`: codificación de variables categóricas.
- `matplotlib.pyplot`: visualización básica (opcional).
- `warnings`: suprime advertencias innecesarias en la ejecución.
- `gc` (garbage collector): optimiza el uso de memoria.



In [ ]:
# Librerías
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
import gc

# Se configura pandas para mostrar todas las columnas
pd.set_option("display.max_columns", None)

# Ignorar advertencias

warnings.filterwarnings("ignore")

# Activar garbage collector para habilitar la recoleccion de basura, esto mejora el rendimiento en ambientes 
# que tienen un uso intensivo de memoria como lo es el caso del manejo de los datasets de la actividad desarrollada

gc.enable()


# Funciones para el preprocesamiento de datos

### Función para contar valores nulos
La función `contar_nulos(data)` permite encontrar valores faltantes en un conjunto de datos. Su objetivo es identificar qué columnas tienen datos incompletos, cuántos valores faltan y qué porcentaje representan respecto al total de registros.

Primero, calcula la cantidad total de valores nulos en cada columna utilizando `data.isnull().sum()`, que cuenta los `True` para cada celda nula. Luego, ordena estas cantidades de forma descendente para priorizar las columnas más problemáticas.

A continuación, calcula el porcentaje de nulos dividiendo cada total de nulos por el número total de filas del `DataFrame`, y multiplica por 100 para expresarlo en porcentaje. También se ordenan los resultados de mayor a menor.

Ambas series (total y porcentaje) se combinan en un solo `DataFrame` usando `pd.concat.

Por último, se filtran las columnas con valores nulos, devolviendo únicamente aquellas que presentan al menos un valor faltante. Esto permite enfocarse exclusivamente en las variables que requieren algún tipo de tratamiento durante la limpieza de datos.

In [4]:

def contar_nulos(data):
    total = data.isnull().sum().sort_values(ascending=False)
    porcentaje = (data.isnull().sum() / len(data) * 100).sort_values(ascending=False)
    tabla = pd.concat([total, porcentaje], axis=1, keys=["Total de nulos", "Porcentaje"])
    return tabla[tabla["Total de nulos"] > 0]


### Función convertir_dias
La función `convertir_dias(data, columnas, divisor=12, redondear=True, reemplazar=False)` se utiliza para transformar columnas que contienen valores de tiempo expresados en días negativos, como por ejemplo DAYS_BIRTH o DAYS_EMPLOYED, en unidades más interpretables como meses o años.

El parámetro columnas especifica la lista de variables que se desean transformar. El parámetro `divisor` determina en qué unidad se hará la conversión (en este caso 12, para convertirlo a meses).

Dentro del bucle, se invierte el signo del valor (porque los días vienen negativos en el dataset), se divide por el `divisor`. Luego, se eliminan los valores negativos resultantes, ya que estos suelen ser errores de codificación y se reemplazan por None.

In [13]:

def convertir_dias(data, columnas, divisor=12, redondear=True, reemplazar=False):
    for col in columnas:
        nueva_columna = "CONVERTIDO_" + str(col)
        valores_convertidos = -data[col] / divisor
        if redondear:
            valores_convertidos = valores_convertidos.round()
        valores_convertidos[valores_convertidos < 0] = None
        if reemplazar:
            data[col] = valores_convertidos
        else:
            data[nueva_columna] = valores_convertidos
    return data


### Función para crear variables logarítmicas
La función `crear_logaritmos(data, columnas, reemplazar=False)` se utiliza para aplicar una transformación logarítmica a una o más variables numéricas del dataset. Este tipo de transformación es común cuando se trabaja con variables con distribuciones altamente sesgadas (como ingresos, montos de crédito, etc.), esto se debe a que las transformaciones logarítmicas pueden ayudar a corregir la asimetría de variables y mejorar la linealidad en modelos estadísticos.

In [6]:

import numpy as np

def crear_logaritmos(data, columnas, reemplazar=False):
    for col in columnas:
        valores_log = np.log(data[col].abs() + 1)
        if reemplazar:
            data[col] = valores_log
        else:
            data["LOG_" + str(col)] = valores_log
    return data


### Función para crear indicadores flags de nulos
``crear_flags_nulos(data, columnas=None)`` tiene como propósito generar nuevas variables binarias (0 o 1) que indiquen si un valor estaba originalmente ausente (nulo) en el dataset. Esto con el fin de que la ausencia de valores se vuelva una informacion relevante para el modelo.
Por ejemplo, si los clientes con ingresos faltantes tienden a incumplir, el simple hecho de no declarar el ingreso se vuelve información sumamente relevante.



In [7]:

def crear_flags_nulos(data, columnas=None):
    if columnas is None:
        columnas = data.columns
    for col in columnas:
        if data[col].isnull().any():
            data["ES_NULO_" + str(col)] = data[col].isnull().astype(int)
    return data


### Función para codificar variables categóricas
Mucha de la información que se tiene del dataset son string o cadenas de texto que el modelo no va a poder interpretar por si solo, se crea la funcion `tratar_categoricas(data)` para que a cada columna que tenga valores de tipo objeto se le reemplace cada categoria por un valor entero unico.
Por ejemplo en la columna `GENRE` se remplazaria ``M`` por 0 y ``F`` por 1

In [ ]:

def tratar_categoricas(data):

    #Codifica todas las variables categóricas del DataFrame usando codificación por etiquetas.
    #Cada categoría única se reemplaza por un número entero único.

    columnas_obj = [col for col in data.columns if data[col].dtype == "object"]
    for col in columnas_obj:
        data[col], _ = pd.factorize(data[col])
    return data



### 📊 Función para calcular tasas de aprobación y rechazo

In [9]:

def calcular_ratio_aprobacion(data, lags=[1, 3, 5]):
    df = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]].copy()
    df["DAYS_DECISION"] = -df["DAYS_DECISION"]
    df = df.sort_values(["SK_ID_CURR", "DAYS_DECISION"])
    df = pd.get_dummies(df)

    for t in lags:
        tmp = df.groupby("SK_ID_CURR")["NAME_CONTRACT_STATUS_Approved"].head(t).groupby("SK_ID_CURR").mean().reset_index()
        tmp.columns = ["SK_ID_CURR", f"RATIO_APROBADO_{t}"]
        data = data.merge(tmp, on="SK_ID_CURR", how="left")

        tmp = df.groupby("SK_ID_CURR")["NAME_CONTRACT_STATUS_Refused"].head(t).groupby("SK_ID_CURR").mean().reset_index()
        tmp.columns = ["SK_ID_CURR", f"RATIO_RECHAZADO_{t}"]
        data = data.merge(tmp, on="SK_ID_CURR", how="left")

    return data


### 📈 Función para agregar datos numéricos y categóricos por cliente

In [10]:

import scipy.stats

def agregar_por_cliente(data, id_col, etiqueta=None):
    print("- Preparando el conjunto de datos...")

    categoricas = [col for col in data.columns if data[col].dtype == "object"]
    datos_numericos = data.drop(columns=categoricas)
    datos_categoricos = data[[id_col] + categoricas]

    print(f"- Variables categóricas: {len(categoricas)}")
    print(f"- Variables numéricas: {datos_numericos.shape[1] - 1}")

    # Agregación numérica
    if datos_numericos.shape[1] > 1:
        print("- Agregando variables numéricas...")
        numericos_agg = datos_numericos.groupby(id_col).agg(["mean", "std", "min", "max"])
        numericos_agg.columns = [f"{col[0]}_{col[1]}" for col in numericos_agg.columns]
        numericos_agg = numericos_agg.sort_index()
    else:
        numericos_agg = pd.DataFrame()

    # Agregación categórica
    if categoricas:
        print("- Agregando variables categóricas...")
        categoricos_agg = datos_categoricos.groupby(id_col).agg({
            col: [lambda x: scipy.stats.mode(x)[0][0], lambda x: x.nunique()]
            for col in categoricas
        })
        categoricos_agg.columns = [f"{col[0]}_{col[1]}" for col in categoricos_agg.columns]
        categoricos_agg = categoricos_agg.sort_index()
    else:
        categoricos_agg = pd.DataFrame()

    # Unir
    if not numericos_agg.empty and not categoricos_agg.empty:
        resultado = pd.concat([numericos_agg, categoricos_agg], axis=1)
    elif not numericos_agg.empty:
        resultado = numericos_agg
    else:
        resultado = categoricos_agg

    if etiqueta:
        resultado.columns = [etiqueta + "_" + col for col in resultado.columns]

    print("- Dimensiones finales:", resultado.shape)
    return resultado
